In [5]:
import pints
import pints.toy

import emupints
import emupints.plot as emuplt
import emupints.utils as emutils

import numpy as np
import matplotlib.pyplot as plt
import GPy
from GPy import kern as kern

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import string
from itertools import product

## Specifying model

In [6]:
# Load problem from predefined ones
model = emupints.Problems.FitzhughNagumoModelDiscontinious
problem = emupints.Problems.load_problem(model)

problem['values'][:5]

array([[-0.87017686,  1.02371723],
       [-1.06067642,  0.9984242 ],
       [-1.17044491,  0.84221808],
       [-0.43302902,  1.00958793],
       [-0.67218723,  1.15831525]])

In [8]:
# take required variables for visualisation
n_parameters = problem['n_parameters']
log_likelihood = problem['log_likelihood']
log_prior = problem['log_prior']
log_posterior = problem['log_posterior']
bounds = problem['bounds']
index_to_param_name = problem['param_names']

## Creating emulator and specifying variables

In [9]:
# training size
training_size = 500

input_parameters = log_prior.sample(training_size)
target_likelihoods = np.apply_along_axis(real_log_likelihood, 1, input_parameters)

emu = emupints.GPEmulator(real_log_likelihood, 
                          input_parameters, 
                          target_likelihoods, 
                          input_scaler=StandardScaler())

## Kernel selection

In [ ]:
n_parameters = emu.n_parameters()

kernels = [
    kern.Linear(n_parameters)
    kern.RBF(n_parameters),
    kern.RatQuad(n_parameters),
    kern.MLP(n_parameters),
    kern.Matern52(n_parameters),
    kern.RatQuad(n_parameters) + kern.RBF(n_parameters) * kern.RBF(n_parameters),
    kern.Matern52(n_parameters) + kern.RBF(n_parameters) * kern.RBF(n_parameters),
    kern.RatQuad(n_parameters) + kern.RBF(n_parameters) + kern.RBF(n_parameters),
    kern.MLP(n_parameters) + kern.RBF(n_parameters) + kern.RBF(n_parameters),
    kern.MLP(n_parameters) + kern.Matern(n_parameters) + kern.RatQuad(n_parameters),
]

kernel_names = [emutils.kernel_to_string(kern)]

# grid optimization of basic kernels
if optimize_hyper_parameters:
    
    variances = [0.1, 1, 10]
    lengthscale = [0.1, 1, 10]

    hyper_kernels = [k(n_parameters, variance=v, lengthscale=l)
                     for k in [kern.RBF, kern.Matern52, kern.RatQuad]  
                     for v in variances
                     for l in lengthscale]
    hyper_kernels.insert(0, kern.Linear(n_parameters))
    hyper_kernels.extend([GPy.kern.MLP(n_parameters, variance = v) for v in variances])
    
    kernels.extend(hyper_kernels)

    kernel_names.extend([emutils.kernel_to_string(kern, decimal_places=2) for kern in hyper_kernels])

# kernels that have been optimized for input data
trained_kernels = []

# possible optimizers: 
# ‘scg’, ‘lbfgs’, ‘tnc’
# can specify max number of iterations using max_iters
optimizer = "lbfgs"
max_iters = 500
emu.set_parameters(optimizer = optimizer)

In [ ]:
# when the output data is normalized values of variance should be small
# hence ignore any kernel that has a subkernel 
# (i.e kernel that is a part of sum/product) 
# with variance > 10000
# set avoid_overfitting to False to stop this effect
avoid_overfitting = False
variance_threshold = 10000

In [ ]:
marginal_likelihoods = []

for kernel, kernel_name in zip(kernels, kernel_names):
    emu.set_parameters(kernel = kernel)
    emu.fit(optimize = False, normalizer = True)
    emu.optimize(max_iters = max_iters, messages = False)
    
    trained_kernel = emu.get_trained_kern()
    trained_kernels.append(trained_kernel)
    
    ml = emu.get_log_marginal_likelihood()
    marginal_likelihoods.append(ml)
    print("{}: {:.2f}".format(kernel_name, ml))

In [ ]:
best_kernel = None
best_score = -1 << 31

# find kernel with highest log marginal likelihood
for kernel, score in zip(trained_kernels, marginal_likelihoods):
    # ignore any overfitting kernel
    if avoid_overfitting and emutils.has_high_variance(kernel, threshold = variance_threshold):
        continue
    # ignore kernels that don't provide the required speed up. at least 5x
    
    if score > best_score:
        best_kernel = kernel
        score = best_score
        
best_kernel

In [ ]:
# when a kernel consists of many additions / multiplications
# utils method kernel_to_string can be useful
print(emutils.kernel_to_string(best_kernel))

In [ ]:
# TODO: directly assign kernel to GP in class
emu.set_parameters(kernel = best_kernel)
emu.fit(optimize = False)

In [ ]:
emu.get_gp()

In [ ]:
if n_parameters == 2:
    # generate data for surfaces
    test_splits = 20 # number of splits along each axis
    r_grid, k_grid, test_data = emutils.generate_grid(bounds.lower(), 
                                                      bounds.upper(), 
                                                      test_splits)    

    emu_grid = emutils.predict_grid(emu, test_data)
    real_grid = emutils.predict_grid(real_log_likelihood, test_data)

    plt.figure(figsize = (10, 5))
    ax = emuplt.surface(r_grid, k_grid, emu_grid, 
                        title = "True log_likelihood",
                        alpha = 0.8,
                        cmap="Blues",
                        x_label = "r (growth rate)",
                        y_label = "k (carrying capacity)"
                       )

    ax.plot_surface(r_grid, k_grid, real_grid, cmap="Reds", alpha = .5)

In [ ]:
if n_parameters >= 3:
    fig, ax = emuplt.plot_fixed_param_grid(
        emu,
        fixed_parameters,
        bounds,
        n_splits = axis_n_splits,
        shape = (n_parameters, n_parameters - 1),
        contour = False,
        additional_log_likelihoods = [real_log_likelihood]
    )

    plt.show(fig)

In [ ]:
np.apply_along_axis(emu, 1, input_parameters).flatten() - target_likelihoods

In [ ]:
# scoring function for CMA-ES and comparison
score = pints.SumOfSquaresError(problem)

# Timing single prediction

In [ ]:
%%timeit
emu(real_parameters)

In [ ]:
%%timeit
real_log_likelihood(real_parameters)

## Running and Timing MCMC

In [ ]:
# for Logistic and Lotka-Voltera use default
mcmc_method = None # Adaptive covariance by default
# mcmc_method = pints.PopulationMCMC
# mcmc_method = pints.MetropolisRandomWalkMCMC
# mcmc_method = pints.DifferentialEvolutionMCMC

# MCMC parameters
num_chains = 3
max_iters = 20000

In [ ]:
emu_posterior = pints.LogPosterior(emu, log_prior)
real_posterior = pints.LogPosterior(real_log_likelihood, log_prior)

In [ ]:
# possible parameter starting points
# use three chains
# substitute for CMA-ES

xs = [
    real_parameters * 0.95,
    real_parameters * 0.90,
    real_parameters * 1.05,
]

In [ ]:
# CMA-ES


In [ ]:
%%time
emu_mcmc = pints.MCMCSampling(emu_posterior, 
                              num_chains, 
                              xs, 
                              method = mcmc_method, 
                             )
emu_mcmc.set_max_iterations(max_iters)
emu_mcmc.set_log_to_screen(False)
print('Running...')
emu_chains = emu_mcmc.run()
print('Done!')

In [ ]:
%%time
# population MCMC
real_mcmc = pints.MCMCSampling(real_posterior, 
                               num_chains, 
                               xs, 
                               method = mcmc_method,
                              )
real_mcmc.set_max_iterations(max_iters)
real_mcmc.set_log_to_screen(False)
# Run!
print('Running...')
real_chains = real_mcmc.run()
print('Done!')

In [ ]:
import pints.plot
pints.plot.trace(emu_chains)
plt.show()

In [ ]:
pints.plot.trace(real_chains)
plt.show()

In [ ]:
# Look at likelihood changes along one chain
chain = emu_chains[0]
emu_prediction = np.apply_along_axis(emu, 1, chain).flatten()
real_prediction = np.apply_along_axis(real_log_likelihood, 1, chain).flatten()

In [ ]:
iters = range(len(chain))
plt.figure(figsize=(10, 5))
plt.xlabel("Number of iterations")
plt.ylabel("Log Likelihood")
plt.plot(iters, emu_prediction, color="Red", label='emu')
plt.plot(iters, real_prediction, color="Blue", label='model')
plt.legend()
plt.show()

In [ ]:
help(pints.MCMCSampling)

In [ ]:
diffs = np.abs(real_prediction - emu_prediction)

In [ ]:
iters = np.linspace(0, 10000, len(chain))
plt.figure(figsize=(10, 5))
plt.xlabel("Number of iterations")
plt.ylabel("Likelihood difference")
plt.plot(iters, diffs, color = "Black")
plt.show()